In [20]:
import numpy as np
import tensorflow as tf 

import os
from sklearn.model_selection import train_test_split
import csv

with open('data/driving_log.csv') as f:
    x = [line for line in csv.reader(f)]
x_train, x_val = train_test_split(x, test_size=0.05)

import matplotlib.image as mpimg
from sklearn.utils import shuffle

PREFIX = './data/';

def gen(x, batch):
    while True:
        shuffle(x)
        for offset in range(0, len(x), batch):
            img_list = []
            metrics_list = []
            for xx in x[offset:offset+batch]:
                steering_center = float(xx[3])
                correction = 0.2 
                steering_left = steering_center + correction
                steering_right = steering_center - correction
                for i in range(3):
                    img = mpimg.imread(PREFIX + xx[i].strip())
                    img_list.append(img)
                metrics_list += [steering_center, steering_left, steering_right]

            yield shuffle(np.array(img_list), np.array(metrics_list))

from keras.layers import Dense, Dropout, Activation, Flatten, ELU
from keras.models import Sequential
from keras.layers import Convolution2D, MaxPooling2D, Cropping2D
from keras.optimizers import Adam
from keras.layers.core import Lambda


def addElu():
    model.add(ELU())
    
def add_flatten_layer():
    model.add(Flatten())
    
def add_conv_layer(dims, subsample, border_mode):
    model.add(Convolution2D(dims[0], dims[1], dims[2], subsample=subsample, border_mode=border_mode))
    
def conv1():
    add_conv_layer((16,8,8), (4,4), "same")
    addElu()
def conv2():
    add_conv_layer((32,5,5), (2,2), "same")
    addElu()
def conv3():
    add_conv_layer((62,5,5), (2,2), "same")
    add_flatten_layer()
    addElu()
    
def crop():
    model.add(Cropping2D(cropping=((70, 25), (0, 0)),
                     dim_ordering='tf', 
                     input_shape=(160, 320, 3)))
    
def resize(img):
    import tensorflow as tf
    return tf.image.resize_images(img, [40, 160])
    
def add_lambdas():
    model.add(Lambda(resize))
    model.add(Lambda(lambda x: (x/255.0) - 0.5))

def add_layer(dims, dropout):
    model.add(Dense(dims))
    if dropout:
        model.add(Dropout(dropout))

def build_and_run_model():
    crop()
    add_lambdas()

    # add conv layers
    conv1()
    conv2()
    conv3()

    add_layer(512, 0.5)
    addElu()
    add_layer(50, False)
    addElu()
    add_layer(1, False)
    model.compile(optimizer=Adam(lr=0.0001), loss="mse", metrics=['accuracy'])
    model.summary()

model = Sequential()
build_and_run_model()
    
model.fit_generator(gen(x_train, 32), 
                    samples_per_epoch=len(x_train) * 3, 
                    validation_data=gen(x_val, 32),
                    nb_val_samples=len(x_val), nb_epoch=5)
model.save('model.h5');

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
cropping2d_18 (Cropping2D)       (None, 65, 320, 3)    0           cropping2d_input_18[0][0]        
____________________________________________________________________________________________________
lambda_36 (Lambda)               (None, 40, 160, 3)    0           cropping2d_18[0][0]              
____________________________________________________________________________________________________
lambda_37 (Lambda)               (None, 40, 160, 3)    0           lambda_36[0][0]                  
____________________________________________________________________________________________________
convolution2d_57 (Convolution2D) (None, 10, 40, 16)    3088        lambda_37[0][0]                  
___________________________________________________________________________________________